# Model recommendation with lighfm

### Import libraries

In [71]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [72]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [73]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [74]:
K_LIGHTFM_ITEMS = 10
K_POPULAR_ITEMS = 0
K_SAMPLED_ITEMS = 0
LIMIT_N_ROWS = 1000
N_PARTS_DASK = 3
SEED = 42

### Retrieve data

In [75]:
import pandas as pd
# path config

# df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid)
df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=LIMIT_N_ROWS)
df_valid.dropna()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
995,0bca64a686750abbb1ba077318fb7d3b094f50c55374a5...,Logged,['a89983c3-e220-4a5f-ab23-5edd8b2b914c'],[1660646489438]
996,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,Logged,['cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2'],[1660591462479]
997,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3'],[1660533586069]
998,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3' 'a41b...,[1660558252865 1660702436161]


In [76]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=LIMIT_N_ROWS)
df_ratings.drop(columns=["Unnamed: 0","score"],inplace=True)
df_ratings

,userId,history,userType
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged
...,...,...,...
995,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,83cf1546-5235-491f-ae0c-90949a2ab3c2,Non-Logged
996,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,2b52a54f-10f5-4d4f-b7a6-bbe48c7e87ae,Non-Logged
997,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,fb96b132-fcde-46cc-86ef-000cb250c361,Non-Logged
998,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,4d89c4b6-6827-4935-9ba1-0502025af270,Non-Logged


In [77]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.122610,very-old


In [78]:
import pickle

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))
loaded_interactions_shape = pickle.load(open('artifacts/interactions_shape.pkl', 'rb'))

loaded_n_users, loaded_n_items = loaded_interactions_shape

### Make predictions to known and unknowm on same recommendation function with pkls

In [79]:
from utils.custom_treat_data_funcs import explode_df_columns

cols_to_explode = ["history", "timestampHistory"]
cols_and_id = cols_to_explode.copy()
cols_and_id.insert(0,"userId")
cols_and_id.append("userType")
cols_and_id = tuple(cols_and_id)
exploded_df = explode_df_columns(df_valid.loc[:,cols_and_id], cols_to_explode)

In [80]:
df_valid

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
995,0bca64a686750abbb1ba077318fb7d3b094f50c55374a5...,Logged,['a89983c3-e220-4a5f-ab23-5edd8b2b914c'],[1660646489438]
996,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,Logged,['cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2'],[1660591462479]
997,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3'],[1660533586069]
998,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3' 'a41b...,[1660558252865 1660702436161]


In [81]:
exploded_df

,userId,history,timestampHistory,userType
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1660533136590,Logged
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,01c59ff6-fb82-4258-918f-2910cb2d4c52,1660672113513,Logged
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,77901133-aee7-4f7b-afc0-652231d76fe9,1660556860253,Logged
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,1660561649242,Logged
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,1660533830245,Logged
...,...,...,...,...
2287,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2,1660591462479,Logged
2288,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3,1660533586069,Logged
2289,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3,1660558252865,Logged
2290,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,a41bed7d-5376-479f-b503-a8b046a4c7e3,1660702436161,Logged


In [82]:
exploded_df = pd.concat([exploded_df, df_ratings]).reset_index()
exploded_df

,index,userId,history,timestampHistory,userType
0,0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1660533136590,Logged
1,1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,01c59ff6-fb82-4258-918f-2910cb2d4c52,1660672113513,Logged
2,2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,77901133-aee7-4f7b-afc0-652231d76fe9,1660556860253,Logged
3,3,755062dd39a48809880cf363b04268c3af2c003088cde0...,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,1660561649242,Logged
4,4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,1660533830245,Logged
...,...,...,...,...,...
3287,995,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,83cf1546-5235-491f-ae0c-90949a2ab3c2,NaN,Non-Logged
3288,996,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,2b52a54f-10f5-4d4f-b7a6-bbe48c7e87ae,NaN,Non-Logged
3289,997,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,fb96b132-fcde-46cc-86ef-000cb250c361,NaN,Non-Logged
3290,998,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,4d89c4b6-6827-4935-9ba1-0502025af270,NaN,Non-Logged


In [83]:
exploded_df["check_hifens"] = exploded_df["history"].str.split('-').apply(lambda x : len(x) == 5).astype("bool")
exploded_df["check_size"] = exploded_df["history"].str.replace(r'-', '', regex=True).apply(lambda x : len(x) == 32).astype("bool")
exploded_df["check_chars"] = exploded_df["history"].str.replace(r'-', '', regex=True).str.replace(r'[a-f0-9]', '', regex=True).apply(lambda x : x == '').astype("bool")
exploded_df["check_history"] = (exploded_df["check_hifens"] & exploded_df["check_size"] & exploded_df["check_chars"]).astype("bool")
exploded_df["check_history"].value_counts()

check_history
True     3205
False      87
Name: count, dtype: int64

In [84]:
exploded_df_valid = exploded_df[~exploded_df["check_history"]==False][["userId","userType","history"]]
exploded_df_valid

,userId,userType,history
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85
...,...,...,...
3287,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,83cf1546-5235-491f-ae0c-90949a2ab3c2
3288,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,2b52a54f-10f5-4d4f-b7a6-bbe48c7e87ae
3289,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,fb96b132-fcde-46cc-86ef-000cb250c361
3290,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,4d89c4b6-6827-4935-9ba1-0502025af270


In [85]:
count_histories = exploded_df_valid.groupby(["history"]).size().sort_values(ascending=False).astype(dtype="UInt16")
count_histories

history
9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6    73
4c3d47a1-6f4b-424f-8944-6c227e686c5c    56
eb23272d-8e6c-479d-b972-eabeb5f6f3dd    52
be89a7da-d9fa-49d4-9fdc-388c27a15bc8    36
82e7b237-e7ea-4cf5-b44c-55385e2cdbb0    26
                                        ..
ff1248ca-02c4-40f6-a292-b8e67862335c     1
ff26f5dd-71d1-4364-940b-8647e048a209     1
ff270579-3662-43ae-9bee-2b398f3611db     1
ff30c8b2-3093-4b84-9ba6-f0fa8d32d7cc     1
ff3e8c61-d72a-42bf-a84a-ecf5409468dd     1
Length: 2165, dtype: UInt16

In [86]:
import random

random.seed(SEED)
rand_counter = 0

def get_pseudo_random_int():
    global rand_counter
    rand_counter = rand_counter + 1
    if rand_counter > 40:
        rand_counter = 0
    return random.randint(0, rand_counter+10)

In [87]:
top_k_popular_histories = count_histories.iloc[:K_POPULAR_ITEMS]
top_k_popular_histories_list = list(set(top_k_popular_histories.keys()))

out_k_pop_histories = count_histories.iloc[K_POPULAR_ITEMS:]
out_k_pop_weights = out_k_pop_histories.values

random_k_histories = out_k_pop_histories.sample(n=K_SAMPLED_ITEMS, weights=out_k_pop_weights, random_state=get_pseudo_random_int())
random_k_histories_list = list(set(random_k_histories.keys()))

print(random_k_histories_list)
print(top_k_popular_histories_list+random_k_histories_list)

[]
[]


In [88]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [89]:
def sample_recommendation_by_title(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LIGHTFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]
    
    random_k_histories = out_k_pop_histories.sample(n=K_SAMPLED_ITEMS, weights=out_k_pop_weights, random_state=get_pseudo_random_int())
    random_k_histories_list = list(set(random_k_histories.keys()))

    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories_list + random_k_histories_list))

    print("Top 5 recommended items:")

    for x in top_k_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["title"].values[0])


In [90]:
def get_recommended_history_list(user_hash,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top 5 recommendations from the list are returned.
    """
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LIGHTFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]

    random_k_histories = out_k_pop_histories.sample(n=K_SAMPLED_ITEMS, weights=out_k_pop_weights, random_state=get_pseudo_random_int())
    random_k_histories_list = list(set(random_k_histories.keys()))
    
    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories_list + random_k_histories_list))

    return top_k_items

In [91]:
# # Testing for just one user
# user = 38
# user_feature_list = [df_valid["userType"].iloc[user]]
# user_hash = df_valid["userId"].iloc[user]
# validation_history_hashes = df_valid["history"].iloc[user]

# recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
# print(recommeded_histories)
# print(validation_history_hashes)
# print(validation_history_hashes in recommeded_histories)

In [92]:
def count_valid_recommendations(validation_history_hashes, recommeded_histories):
    """
    This function receives 
    * The `validation_history_hashes` (which is a list of histories contained in the "validacao.csv")
    * The `recommeded_histories` (which are the recommended histories/items by the model)
    Then, it verifies how many recommended items match with the validation histories.
    """
    count_valids = 0
    for valid_hist in validation_history_hashes:
        if valid_hist in recommeded_histories:
            count_valids = count_valids+1
    return count_valids

In [93]:
def check_valid_recommendations(validation_history_hash, recommeded_histories):
    """
    Check if validation_history_hash is in recommeded_histories
    """
    # print(validation_history_hash)
    # print(recommeded_histories)
    recommendation_in_validation_flag = False
    if validation_history_hash in recommeded_histories:
        recommendation_in_validation_flag = True
    return recommendation_in_validation_flag
    

In [94]:
# for index, row in df_valid.iterrows():
#     """
#     For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
#     Stores the results on the columns `recommended_hists` `matched_recommendations`.
#     """
#     user_feature_list = [row["userType"]]
#     user_hash = row["userId"]
#     validation_history_hashes = row["history"]

#     recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
#     num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

#     row["recommended_hists"] = recommended_hist
#     row["matched_recommendations"] = num_valid_recommendations

In [95]:
# # exploded_df_valid = exploded_df_valid.head(1000)

# exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.apply(lambda x :
#     get_recommended_history_list(
#         x.userId,
#         [x.userType],
#         loaded_item_id_map_reverse,
#         loaded_user_feature_map,
#         loaded_user_id_map,
#         loaded_model),
#         axis=1)

In [96]:
# import swifter

# exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.swifter.apply(lambda x :
#     get_recommended_history_list(
#         x.userId,
#         [x.userType],
#         loaded_item_id_map_reverse,
#         loaded_user_feature_map,
#         loaded_user_id_map,
#         loaded_model),
#         axis=1)

In [97]:
# import multiprocessing
# from joblib import Parallel, delayed

# results = Parallel(n_jobs=multiprocessing.cpu_count())(
#     exploded_df_valid.apply(lambda x :
#         get_recommended_history_list(
#             x.userId,
#             [x.userType],
#             loaded_item_id_map_reverse,
#             loaded_user_feature_map,
#             loaded_user_id_map,
#             loaded_model),
#             axis=1)
#             )
# exploded_df_valid.loc[:,"recommended_hists"] = results

In [98]:
# import dask.dataframe as dd
# from dask.multiprocessing import get

# ddata = dd.from_pandas(exploded_df_valid, npartitions=3)

# def apply_my_func_to_df(df):
#     df.apply(lambda x :
#         get_recommended_history_list(
#             x.userId,
#             [x.userType],
#             loaded_item_id_map_reverse,
#             loaded_user_feature_map,
#             loaded_user_id_map,
#             loaded_model),
#             axis=1)

# result = ddata.map_partitions(apply_my_func_to_df).compute(get=get)

In [99]:
import dask.dataframe as dd
import dask.multiprocessing

ddf_exploded_valid = dd.from_pandas(exploded_df_valid, npartitions=N_PARTS_DASK)

result = ddf_exploded_valid.apply(lambda x :
    get_recommended_history_list(
        x.userId,
        [x.userType],
        loaded_item_id_map_reverse,
        loaded_user_feature_map,
        loaded_user_id_map,
        loaded_model),
        axis=1,
        meta=('string','string')).compute()

exploded_df_valid["recommended_hists"] = result


In [100]:
# list_ex_0 = exploded_df_valid.loc[1,"recommended_hists"]
# list_ex_0.append(exploded_df_valid.loc[1,"history"])
# list_to_check = list(set(list_ex_0))

# test_history = exploded_df_valid.loc[1,"history"]
# print(test_history)
# print()
# check_valid_recommendations(test_history, list_to_check)

In [101]:
exploded_df_valid["matched_recommendations"] = exploded_df_valid.apply(lambda x :
    check_valid_recommendations(x.history, x.recommended_hists),
    axis=1
    )

In [102]:
exploded_df_valid

,userId,userType,history,recommended_hists,matched_recommendations
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,"[20fcadfe-49c3-4c9e-b03a-04b9ff8eca9f, f03509a...",False
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,"[20fcadfe-49c3-4c9e-b03a-04b9ff8eca9f, f03509a...",False
...,...,...,...,...,...
3287,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,83cf1546-5235-491f-ae0c-90949a2ab3c2,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
3288,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,2b52a54f-10f5-4d4f-b7a6-bbe48c7e87ae,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
3289,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,fb96b132-fcde-46cc-86ef-000cb250c361,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False
3290,54a1dcc6afbc8547ffef82bf9a9da7491c3af226752853...,Non-Logged,4d89c4b6-6827-4935-9ba1-0502025af270,"[707c5b71-c932-4c2e-960c-6245a41cbd20, c9a66c0...",False


In [103]:
# user = 1
# user_feature_list = [exploded_df_valid["userType"].iloc[user]]
# user_hash = exploded_df_valid["userId"].iloc[user]
# validation_history_hashes = exploded_df_valid["history"].iloc[user]

# print(user_hash)
# print(user_feature_list)

# recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
# print(recommeded_histories)
# print(validation_history_hashes)
# print(validation_history_hashes in recommeded_histories)

In [104]:
exploded_df_valid[exploded_df_valid["matched_recommendations"]==True]

,userId,userType,history,recommended_hists,matched_recommendations


In [105]:
exploded_df_valid[exploded_df_valid["matched_recommendations"]==True]["userId"].count()

np.int64(0)